In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import cPickle as pickle

plt.style.use('ggplot')

In [2]:
cd ..

/Users/odatakuma/cmu/FleetAI


In [3]:
data_path = 'analytics/data/taxi_trips/taxi_tripdata_2016-05.csv'

In [14]:
cols = ['dayofweek', 'hour', 'minute', 'pickup_latitude', 'pickup_longitude',
        'dropoff_latitude', 'dropoff_longitude', 'great_circle_distance', 'trip_time']
df = pd.read_csv(data_path, usecols=cols, dtype={key : np.float32 for key in cols})
df.head()

,dayofweek,dropoff_latitude,dropoff_longitude,hour,minute,pickup_latitude,pickup_longitude,trip_time,great_circle_distance
0,6,40.730099,-73.983986,0,0,40.768040,-73.985901,17.516666,4221
1,6,40.744633,-73.980995,0,0,40.741573,-73.993073,7.016667,1072
2,6,40.733002,-74.002258,0,0,40.684601,-73.991943,19.783333,5451
3,6,40.765469,-73.975700,0,0,40.744751,-73.991577,7.516667,2663
4,6,40.737564,-73.997498,0,0,40.740192,-74.005280,6.650000,717


In [15]:
df.shape

(12912396, 9)

In [16]:
df['hour'] = df.hour + df.minute/60
df = df[(df.trip_time>1.0) & (df.trip_time<60*3)]
df = df[(df.great_circle_distance>100) & (df.great_circle_distance<100000)]
df['great_circle_speed'] = df['great_circle_distance'] / df.trip_time / 1000 * 60
df = df[df.great_circle_speed > 2]
df = df[df.great_circle_speed < 150]
df = df.drop(['minute', 'great_circle_speed'], axis=1)
df.shape
# df['trip_distance'] = df.trip_distance * 1.60934
# df = df[df.trip_distance > df.great_circle_distance]
# df = df[df.trip_distance < df.great_circle_distance*5]

(12869846, 8)

In [17]:
from sklearn.cross_validation import train_test_split

X = df[['dayofweek', 'hour', 'pickup_latitude', 'pickup_longitude',
        'dropoff_latitude', 'dropoff_longitude', 'great_circle_distance']].values
y = df['trip_time'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print X.shape

(12869846, 7)


In [8]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(learning_rate=1.0, n_estimators=20, subsample=0.2, max_depth=4)
gbr.fit(X_train, y_train)
rmse_train = np.sqrt(((y_train - gbr.predict(X_train))**2).mean())
rmse_test = np.sqrt(((y_test - gbr.predict(X_test))**2).mean())
print "RMSE train/val: %.1f / %.1f" % (rmse_train, rmse_test)

RMSE train/val: 5.3 / 5.3
CPU times: user 6min 40s, sys: 9.05 s, total: 6min 49s
Wall time: 6min 56s


In [13]:
%%time
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=10, min_samples_split=2e-5, n_jobs=-1)
forest.fit(X_train, y_train)
rmse_train = np.sqrt(((y_train - forest.predict(X_train))**2).mean())
rmse_test = np.sqrt(((y_test - forest.predict(X_test))**2).mean())
print "RMSE train/val: %.1f / %.1f" % (rmse_train, rmse_test)

RMSE train/val: 4.3 / 4.6
CPU times: user 27min 49s, sys: 6.93 s, total: 27min 56s
Wall time: 8min 29s


In [17]:
from keras.models import Sequential
from keras.layers.core import Dense

def MLP(size, act='relu'):
    model = Sequential()
    model.add(Dense(size, input_dim=7, activation=act))
    model.add(Dense(size, activation=act))
    model.add(Dense(size, activation=act))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error',  optimizer='adam')
    return model

In [18]:
%%time
mlp = MLP(64)
mlp.fit(X_train, y_train, nb_epoch=10, batch_size=1000, verbose=0, validation_split=0.0)
y_pred_train = mlp.predict(X_train, batch_size=10000, verbose=0)[:, 0]
y_pred_test = mlp.predict(X_test, batch_size=10000, verbose=0)[:, 0]
rmse_train = np.sqrt(((y_train - y_pred_train)**2).mean())
rmse_test = np.sqrt(((y_test - y_pred_test)**2).mean())
print "RMSE train/val: %.1f / %.1f" % (rmse_train, rmse_test)                         

RMSE train/val: 5.7 / 5.7
CPU times: user 19min 26s, sys: 2min 24s, total: 21min 50s
Wall time: 8min 43s


In [18]:
from sklearn.ensemble import RandomForestRegressor

In [19]:
%%time
forest = RandomForestRegressor(n_estimators=10, min_samples_split=5e-5, n_jobs=-1)
forest.fit(X_train, y_train)
rmse_train = np.sqrt(((y_train - forest.predict(X_train))**2).mean())
rmse_test = np.sqrt(((y_test - forest.predict(X_test))**2).mean())
print "RMSE train/val: %.1f / %.1f" % (rmse_train, rmse_test)

RMSE train/val: 4.9 / 5.0
CPU times: user 27min 42s, sys: 7.51 s, total: 27min 50s
Wall time: 8min 28s


In [3]:
path = 'engine/model/pickle/triptime_predictor.pkl'

In [21]:
with open(path, 'wb') as f:
    pickle.dump(forest, f)

In [4]:
with open(path, 'r') as f:
    model = pickle.load(f)

In [7]:
geohash_table = pd.read_csv('engine/data/zones.csv', index_col='geohash')
geohash_table.head()

,lat,lon,taxi_zone,road_density,intxn_density,latest_demand,x,y
geohash,,,,,,,,
dr5qgxx,40.603358,-74.015682,10,56,36,0,3,0
dr5qgxy,40.604832,-74.017029,10,43,8,0,3,0
dr5qgxz,40.604637,-74.015335,10,22,0,0,3,0
dr5qgz7,40.602178,-74.010322,10,28,9,0,4,0
dr5qgz8,40.603474,-74.014180,10,59,14,0,3,0


In [16]:
df = geohash_table.groupby('taxi_zone')[['lat', 'lon']].mean()
df.index = np.array(df.index).astype(int)
df.head()

,lat,lon
2,40.865465,-73.849574
3,40.724116,-73.977340
6,40.761576,-73.919734
7,40.778363,-73.922796
8,40.753302,-73.788666


In [17]:
data = pd.DataFrame(index=[(p, d) for p in df.index for d in df.index], columns=['plat', 'plon', 'dlat', 'dlon'])
data.loc[[(p, d) for p in df.index for d in df.index], :] = [(plat, plon, dlat, dlon) for plat, plon in df.values
                                                           for dlat, dlon in df.values]
data['dayofweek'] = 0
data['hour'] = 0
data.head()

,plat,plon,dlat,dlon,dayofweek,hour
"(2, 2)",40.865465,-73.849574,40.865465,-73.849574,0,0
"(2, 3)",40.865465,-73.849574,40.724116,-73.977340,0,0
"(2, 6)",40.865465,-73.849574,40.761576,-73.919734,0,0
"(2, 7)",40.865465,-73.849574,40.778363,-73.922796,0,0
"(2, 8)",40.865465,-73.849574,40.753302,-73.788666,0,0


In [19]:
from engine.mapper import geohelper as gh
data['distance'] = gh.distance_in_meters(data.plat, data.plon, data.dlat, data.dlon)

In [24]:
%%time
taxi_zones = df.index
nzones = len(taxi_zones)
df_od = pd.DataFrame(index=[(d, h, z) for d in range(7) for h in range(24) for z in taxi_zones],
                    columns=taxi_zones)

for d in range(7):
    for h in range(24):
        df_od.loc[[(d, h, z) for z in taxi_zones], :] = model.predict(data.values).reshape((nzones, nzones))
df_od.head()

CPU times: user 44 s, sys: 4.91 s, total: 49 s
Wall time: 1min 6s


In [25]:
df_od.head()

,2,3,6,7,8,9,10,11,12,13,...,253,254,255,256,257,258,259,260,261,262
"(0, 0, 2)",2.38634,42.004,28.5031,30.4662,36.6025,41.1861,42.004,42.004,42.004,42.004,...,12.2993,42.004,42.004,42.004,41.1861,10.4077,41.1861,42.004,32.5984,37.8962
"(0, 0, 3)",41.1861,2.32353,20.3072,20.6073,41.1861,41.1861,33.6279,21.3271,12.6219,33.4442,...,41.1861,9.23845,16.03,24.9837,18.5253,41.1861,20.3072,12.6219,21.3085,20.3072
"(0, 0, 6)",18.6067,25.6595,2.38634,8.26668,18.5253,41.1861,42.004,33.0934,32.4451,42.004,...,41.1861,21.6977,20.5591,33.6279,18.3099,41.1861,16.03,32.4451,9.75905,10.6559
"(0, 0, 7)",18.6067,29.8051,8.5416,2.38634,19.7159,41.1861,42.004,33.4442,33.2517,42.004,...,19.0614,20.3072,21.4956,42.004,19.7159,41.1861,17.9234,33.4442,16.8297,22.9587
"(0, 0, 8)",36.4144,42.004,27.9968,30.4662,2.38634,16.4247,42.004,42.004,42.004,42.004,...,41.1861,42.004,42.004,42.004,18.3099,41.1861,24.5827,42.004,42.004,42.004


In [27]:
for key in ['dayofweek', 'hour', 'pickup_zone']:
    df_od[key] = 0
df_od.loc[:, ['dayofweek', 'hour', 'pickup_zone']] = list(df_od.index)
df_od = df_od.reset_index().drop('index', axis=1)
df_od = df_od.set_index(['dayofweek', 'hour', 'pickup_zone'])
df_od.head()

2        3        6        7        8    \
dayofweek hour pickup_zone                                                
0         0    2            2.38634   42.004  28.5031  30.4662  36.6025   
               3            41.1861  2.32353  20.3072  20.6073  41.1861   
               6            18.6067  25.6595  2.38634  8.26668  18.5253   
               7            18.6067  29.8051   8.5416  2.38634  19.7159   
               8            36.4144   42.004  27.9968  30.4662  2.38634   

                                9        10       11       12       13   \
dayofweek hour pickup_zone                                                
0         0    2            41.1861   42.004   42.004   42.004   42.004   
               3            41.1861  33.6279  21.3271  12.6219  33.4442   
               6            41.1861   42.004  33.0934  32.4451   42.004   
               7            41.1861   42.004  33.4442  33.2517   42.004   
               8            16.4247   42.004   42.004   42.004   42.004   

                             ...         253      254      255      256  \
dayofweek hour pickup_zone   ...                                          
0         0    2             ...     12.2993   42.004   42.004   42.004   
               3             ...     41.1861  9.23845    16.03  24.9837   
               6             ...     41.1861  21.6977  20.5591  33.6279   
               7             ...     19.0614  20.3072  21.4956   42.004   
               8             ...     41.1861   42.004   42.004   42.004   

                                257      258      259      260      261  \
dayofweek hour pickup_zone                                                
0         0    2            41.1861  10.4077  41.1861   42.004  32.5984   
               3            18.5253  41.1861  20.3072  12.6219  21.3085   
               6            18.3099  41.1861    16.03  32.4451  9.75905   
               7            19.7159  41.1861  17.9234  33.4442  16.8297   
               8            18.3099  41.1861  24.5827   42.004   42.004   

                                262  
dayofweek hour pickup_zone           
0         0    2            37.8962  
               3            20.3072  
               6            10.6559  
               7            22.9587  
               8             42.004  

[5 rows x 221 columns]

In [28]:
df_od.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 37128 entries, (0, 0, 2) to (6, 23, 262)
Columns: 221 entries, 2 to 262
dtypes: object(221)
memory usage: 62.9+ MB


In [29]:
df_od.to_csv('engine/data/eta.csv')